## Prompt Chaining

Prompt Chaining is a technique in LLM workflows where you break a complex task into multiple smaller steps, each handled by its own prompt.
The output of one prompt becomes the input to the next, forming a chain.

#### 🧠 Why use Prompt Chaining?

LLMs struggle with long, multi-step reasoning in a single prompt.
By chaining prompts, you:

| Benefit              | Meaning                              |
| -------------------- | ------------------------------------ |
| Decompose complexity | Handle one step at a time            |
| Improve accuracy     | Fewer hallucinations                 |
| Add structure        | Each step has a clear responsibility |
| Enable modularity    | Easy to replace or improve one step  |


Prompt 1 → Output A → Prompt 2 → Output B → Prompt 3 → Final Result

### Import the required libraries

In [6]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

1️⃣ Initialize model

In [7]:

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4o-mini"

#### Sequence of Prompts

2️⃣ Split report into manageable sections

In [8]:

def split_report(text, max_length=1000):
    words = text.split()
    sections, current, length = [], [], 0
    for w in words:
        if length + len(w) + 1 > max_length:
            sections.append(" ".join(current))
            current, length = [], 0
        current.append(w)
        length += len(w) + 1
    if current:
        sections.append(" ".join(current))
    return sections


3️⃣ Summarize one section

In [9]:
def summarize_section(section):
    prompt_template = PromptTemplate.from_template("""
    You are a market research analyst.
    Summarize the following section into 3–4 bullet points
    focusing on growth trends, numbers, and insights.

    Section:
    {section}
    """)
    prompt = prompt_template.format(section=section)
    response = llm.invoke(prompt)
    return response.content


4️⃣ Answer business questions using summaries

In [10]:
def answer_business_question(summaries, question):
    context = "\n\n".join(summaries)
    prompt_template = PromptTemplate.from_template("""
    You are a strategy consultant.
    Using the following research summaries, answer the question
    with concise and factual reasoning.

    Context:
    {context}

    Question:
    {question}
    """)
    prompt = prompt_template.format(context=context, question=question)
    response = llm.invoke(prompt)
    return response.content


5️⃣ Full chaining pipeline

In [11]:
def analyze_report(report_text, questions):
    sections = split_report(report_text)
    print(f"📘 Report split into {len(sections)} sections.")

    summaries = []
    for i, sec in enumerate(sections):
        summary = summarize_section(sec)
        summaries.append(summary)
        print(f"🧾 Section {i+1} summarized.")

    results = []
    for q in questions:
        ans = answer_business_question(summaries, q)
        results.append((q, ans))
    return results

In [12]:
market_report = """
India’s EV market is expanding rapidly, projected to reach $150 billion by 2030.
Key drivers include government subsidies, rising fuel costs, and sustainability focus.
Two-wheelers and three-wheelers dominate current adoption, while four-wheelers are growing fast.
Battery recycling and domestic cell manufacturing are emerging opportunities.
Challenges include high upfront costs and limited charging infrastructure.
"""

questions = [
    "What are the main growth drivers for the EV market?",
    "What challenges could slow down the expansion?",
    "Which sub-segments show the highest growth potential?"
]

In [13]:
results = analyze_report(market_report, questions)

📘 Report split into 1 sections.
🧾 Section 1 summarized.


7️⃣ Display results

In [14]:

for q, a in results:
    print(f"\n❓ {q}\n💡 {a}")


❓ What are the main growth drivers for the EV market?
💡 The main growth drivers for the EV market in India are:

1. **Government Subsidies**: These incentives reduce the cost burden on consumers and manufacturers, making electric vehicles (EVs) more financially accessible and attractive.

2. **Rising Fuel Costs**: Increasing costs of conventional fuels make EVs a more economical choice for consumers over the long term, encouraging the shift from internal combustion engines to electric alternatives.

3. **Focus on Sustainability**: Growing environmental awareness and commitments to sustainability are motivating both consumers and policymakers to prioritize cleaner transportation options like EVs.

4. **Emerging Opportunities in Battery Recycling and Domestic Cell Manufacturing**: These opportunities support the ecosystem of the EV market, reducing dependency on imports, lowering costs, and improving the overall efficiency and sustainability of the sector.

❓ What challenges could slow 

### LangChain Style Example

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

prompt1 = PromptTemplate.from_template("Summarize this text:\n{text}")
prompt2 = PromptTemplate.from_template("Extract keywords from this summary:\n{summary}")

chain = prompt1 | llm | prompt2 | llm

result = chain.invoke({"text": market_report})
print(result)


content='- India\n- Electric Vehicle (EV) Market\n- Rapid Growth\n- $150 Billion\n- 2030\n- Government Subsidies\n- Fuel Prices\n- Sustainability\n- Two-wheelers\n- Three-wheelers\n- Four-wheelers\n- Battery Recycling\n- Domestic Cell Manufacturing\n- High Upfront Costs\n- Limited Charging Infrastructure' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 384, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-ChtCpTXEWIPHW5TZaI49Alw27E2og', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--384f4fcc-379d-4582-ba8b-f1233d905083-0' usage_metadata={'input_tokens': 384, 'output_tokens': 71, 'total_tokens': 45

In [16]:
result

AIMessage(content='- India\n- Electric Vehicle (EV) Market\n- Rapid Growth\n- $150 Billion\n- 2030\n- Government Subsidies\n- Fuel Prices\n- Sustainability\n- Two-wheelers\n- Three-wheelers\n- Four-wheelers\n- Battery Recycling\n- Domestic Cell Manufacturing\n- High Upfront Costs\n- Limited Charging Infrastructure', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 384, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-ChtCpTXEWIPHW5TZaI49Alw27E2og', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--384f4fcc-379d-4582-ba8b-f1233d905083-0', usage_metadata={'input_tokens': 384, 'output_tokens': 71, 'tot